In [1]:
import pandas as pd
import gffutils
import pybedtools
import re
import numpy as np

%load_ext autoreload
%autoreload 2

from poshsplice.junctions_to_events import make_junction_exon_triples

v19db_filename = '/projects/ps-yeolab/genomes/hg19/gencode/v19/gencode.v19.annotation.gtf.db'
v19db = gffutils.FeatureDB(v19db_filename)

folder = '/home/obotvinnik/projects/singlecell_pnms/analysis/csvs_for_paper'

sj_metadata = pd.read_csv('{}/sj_metadata.csv'.format(folder), index_col=0)
sj_metadata.head()

/home/obotvinnik/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:784: UserWarning: Found matplotlib configuration in ~/.matplotlib/. To conform with the XDG base directory standard, this configuration location has been deprecated on Linux, and the new location is now /home/obotvinnik/.config/matplotlib/. Please move your configuration there to ensure that matplotlib will continue to find it in the future.
  _get_xdg_config_dir())


,chrom,intron_start,intron_stop,strand,intron_motif,annotated
intron_location,,,,,,
chr1:14830-14969:-,chr1,14830,14969,-,GT/AG,True
chr1:15039-15795:-,chr1,15039,15795,-,GT/AG,True
chr1:228777-237053:-,chr1,228777,237053,-,GT/AG,True
chr1:320654-324287:+,chr1,320654,324287,+,GT/AG,True
chr1:320939-321031:+,chr1,320939,321031,+,GT/AG,True


In [ ]:
# sj_metadata['exon_stop'] = sj_metadata['intron_start'] - 1
# sj_metadata['exon_start'] = sj_metadata['intron_stop'] + 1
# sj_metadata.head()

In [ ]:
junction_exon_triples = make_junction_exon_triples(sj_metadata, v19db)

In [ ]:
range(3)

In [7]:
junction_exon_triples.head()

,exon,direction,junction
0,exon:chr1:29321-29370:-,downstream,chr1:18367-29320:-
1,exon:chr1:29321-29370:-,downstream,chr1:24892-29320:-
2,exon:chr1:29321-29370:-,downstream,chr1:27186-29320:-
3,exon:chr1:29321-29370:-,downstream,chr1:18380-29320:-
4,exon:chr1:29321-29370:-,downstream,chr1:18370-29320:-


In [9]:
junction_exon_triples.to_csv('{}/junction_exon_triples.csv'.format(folder), index=False)

In [10]:
import poshsplice

In [11]:
poshsplice.wannabe()

### Convert `strand`, `intron_motif`, `chrom` to object

In [3]:
sj_metadata.intron_motif = pd.Categorical(sj_metadata.intron_motif, categories=sj_metadata.intron_motif.unique())
sj_metadata.strand = pd.Categorical(sj_metadata.strand, categories=['+', '-'], ordered=True)

chrom_numbers = map(str, list(range(1, 23)) + ['X', 'Y', 'M'])
chrom_categories = ['chr{}'.format(x) for x in chrom_numbers]
sj_metadata.chrom = pd.Categorical(sj_metadata.chrom, categories=chrom_categories, ordered=True)
sj_metadata.head()

,chrom,intron_start,intron_stop,strand,intron_motif,annotated,exon_stop,exon_start
intron_location,,,,,,,,
chr1:14830-14969:-,chr1,14830,14969,-,GT/AG,True,14829,14970
chr1:15039-15795:-,chr1,15039,15795,-,GT/AG,True,15038,15796
chr1:228777-237053:-,chr1,228777,237053,-,GT/AG,True,228776,237054
chr1:320654-324287:+,chr1,320654,324287,+,GT/AG,True,320653,324288
chr1:320939-321031:+,chr1,320939,321031,+,GT/AG,True,320938,321032


In [4]:
n_exons = sum(1 for _ in v19db.features_of_type('exon'))
print n_exons

562680


In [13]:
%%time

# sj_metadata['upstream_exon'] = ''
# sj_metadata['downstream_exon'] = ''

data = []

print 'Starting annotation of all junctions with known exons...'
for i, exon in enumerate(v19db.features_of_type('exon')):
    if i > 20:
        break
#     if (i+1) % 10000 == 0:
#         print '\t{}/{}'.format(i+1, n_exons)
    chrom_ind = sj_metadata.chrom == exon.chrom
    strand_ind = sj_metadata.strand == exon.strand
    upstream_ind = chrom_ind & strand_ind & (sj_metadata.exon_stop == exon.stop)
    downstream_ind = chrom_ind & strand_ind & (sj_metadata.exon_start == exon.start)
    
    exon_id = exon.id
    
    if upstream_ind.any():
        print i, 'upstream'
        print upstream_ind[upstream_ind]
        break
    if downstream_ind.any():
        print i, 'downstream'
        print downstream_ind[downstream_ind]
        break

#     upstream_df = pd.DataFrame([])
    
#     if upstream_ind.any():
#         if exon.strand == '+':
#             sj_metadata.loc[upstream_ind, 'upstream_exon'] = sj_metadata.loc[upstream_ind, 'upstream_exon'] + ',' + exon_id
#         else:
#             sj_metadata.loc[upstream_ind, 'downstream_exon'] = sj_metadata.loc[upstream_ind, 'downstream_exon'] + ',' + exon_id

#     if downstream_ind.any():
#         if exon.strand == '+':
#             sj_metadata.loc[downstream_ind, 'downstream_exon'] = sj_metadata.loc[downstream_ind, 'downstream_exon'] + ',' + exon_id
#         else:
#             sj_metadata.loc[downstream_ind, 'upstream_exon'] = sj_metadata.loc[downstream_ind, 'upstream_exon'] + ',' + exon_id
# print 'Done.'

Starting annotation of all junctions with known exons...
15 downstream
intron_location
chr1:18367-29320:-    True
chr1:24892-29320:-    True
chr1:27186-29320:-    True
chr1:18380-29320:-    True
chr1:18370-29320:-    True
dtype: bool
CPU times: user 1.22 s, sys: 2 ms, total: 1.22 s
Wall time: 1.22 s


In [14]:
downstream_ind[downstream_ind]

intron_location
chr1:18367-29320:-    True
chr1:24892-29320:-    True
chr1:27186-29320:-    True
chr1:18380-29320:-    True
chr1:18370-29320:-    True
dtype: bool

In [20]:
exon_id

'exon:chr1:29321-29370:-'

In [21]:
import itertools
pd.DataFrame(zip(itertools.cycle((exon_id,)),
                 itertools.cycle(('downstream',)), 
                 downstream_ind[downstream_ind].index), 
             columns=['exon', 'direction', 'junction'])

,junction,direction,exon
0,chr1:18367-29320:-,downstream,exon:chr1:29321-29370:-
1,chr1:24892-29320:-,downstream,exon:chr1:29321-29370:-
2,chr1:27186-29320:-,downstream,exon:chr1:29321-29370:-
3,chr1:18380-29320:-,downstream,exon:chr1:29321-29370:-
4,chr1:18370-29320:-,downstream,exon:chr1:29321-29370:-


In [ ]:
%%time

sj_metadata['upstream_exon'] = ''
sj_metadata['downstream_exon'] = ''

print 'Starting annotation of all junctions with known exons...'
for i, exon in enumerate(v19db.features_of_type('exon')):
    if (i+1) % 10000 == 0:
        print '\t{}/{}'.format(i+1, n_exons)
    chrom_ind = sj_metadata.chrom == exon.chrom
    strand_ind = sj_metadata.strand == exon.strand
    upstream_ind = chrom_ind & strand_ind & (sj_metadata.exon_stop == exon.stop)
    downstream_ind = chrom_ind & strand_ind & (sj_metadata.exon_start == exon.start)
    
    exon_id = exon.id
    if upstream_ind.any():
        if exon.strand == '+':
            sj_metadata.loc[upstream_ind, 'upstream_exon'] = sj_metadata.loc[upstream_ind, 'upstream_exon'] + ',' + exon_id
        else:
            sj_metadata.loc[upstream_ind, 'downstream_exon'] = sj_metadata.loc[upstream_ind, 'downstream_exon'] + ',' + exon_id

    if downstream_ind.any():
        if exon.strand == '+':
            sj_metadata.loc[downstream_ind, 'downstream_exon'] = sj_metadata.loc[downstream_ind, 'downstream_exon'] + ',' + exon_id
        else:
            sj_metadata.loc[downstream_ind, 'upstream_exon'] = sj_metadata.loc[downstream_ind, 'upstream_exon'] + ',' + exon_id
print 'Done.'

In [ ]:
sj_metadata.head()

In [ ]:
sj_metadata.tail()

Intermediate saving

In [ ]:
mkdir /projects/ps-yeolab/obotvinnik/singlecell_pnms

In [ ]:
folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'

In [16]:
sj_metadata.to_csv('{}/sj_metadata_exons.csv'.format(folder))

## Create `(exon, direction, junction)` triples

In [2]:
import pandas as pd
import gffutils
import pybedtools
import re
import numpy as np

v19db_filename = '/projects/ps-yeolab/genomes/hg19/gencode/v19/gencode.v19.annotation.gtf.db'
v19db = gffutils.FeatureDB(v19db_filename)

folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'

sj_metadata = pd.read_csv('{}/sj_metadata_exons.csv'.format(folder), index_col=0)
sj_metadata.head()

,chrom,intron_start,intron_stop,strand,intron_motif,annotated,intron_location,exon_stop,exon_start,exon_5p,exon_3p
0,chr1,1309283,1309379,-,GT/AG,True,chr1:1309283-1309379:-,1309282,1309380,exon:chr1:1309380-1309825:-,"exon:chr1:1309110-1309282:-,exon:chr1:1309112-..."
1,chr1,1309826,1310084,-,GT/AG,True,chr1:1309826-1310084:-,1309825,1310085,"exon:chr1:1310085-1310537:-,exon:chr1:1310085-...",exon:chr1:1309380-1309825:-
2,chr1,1310171,1310533,-,GT/AG,True,chr1:1310171-1310533:-,1310170,1310534,exon:chr1:1310534-1310875:-,"exon:chr1:1310085-1310170:-,exon:chr1:1309624-..."
3,chr1,1337637,1341188,-,GT/AG,True,chr1:1337637-1341188:-,1337636,1341189,"exon:chr1:1341189-1341266:-,exon:chr1:1341189-...","exon:chr1:1337288-1337636:-,exon:chr1:1337293-..."
4,chr1,1341267,1342288,-,GT/AG,True,chr1:1341267-1342288:-,1341266,1342289,"exon:chr1:1342289-1342399:-,exon:chr1:1342289-...","exon:chr1:1341189-1341266:-,exon:chr1:1340489-..."


In [3]:
# sj_metadata.loc[sj_metadata.index.map(lambda x: x.endswith('5p')), 'exon_3p'] = np.nan
# sj_metadata.loc[sj_metadata.index.map(lambda x: x.endswith('3p')), 'exon_5p'] = np.nan

In [4]:
sj_metadata.head()

,chrom,intron_start,intron_stop,strand,intron_motif,annotated,intron_location,exon_stop,exon_start,exon_5p,exon_3p
0,chr1,1309283,1309379,-,GT/AG,True,chr1:1309283-1309379:-,1309282,1309380,exon:chr1:1309380-1309825:-,"exon:chr1:1309110-1309282:-,exon:chr1:1309112-..."
1,chr1,1309826,1310084,-,GT/AG,True,chr1:1309826-1310084:-,1309825,1310085,"exon:chr1:1310085-1310537:-,exon:chr1:1310085-...",exon:chr1:1309380-1309825:-
2,chr1,1310171,1310533,-,GT/AG,True,chr1:1310171-1310533:-,1310170,1310534,exon:chr1:1310534-1310875:-,"exon:chr1:1310085-1310170:-,exon:chr1:1309624-..."
3,chr1,1337637,1341188,-,GT/AG,True,chr1:1337637-1341188:-,1337636,1341189,"exon:chr1:1341189-1341266:-,exon:chr1:1341189-...","exon:chr1:1337288-1337636:-,exon:chr1:1337293-..."
4,chr1,1341267,1342288,-,GT/AG,True,chr1:1341267-1342288:-,1341266,1342289,"exon:chr1:1342289-1342399:-,exon:chr1:1342289-...","exon:chr1:1341189-1341266:-,exon:chr1:1340489-..."


### Get transcripts which contain these junctions

In [5]:
sj_with_exons = sj_metadata.dropna(subset=['exon_5p', 'exon_3p'], how='all')
sj_with_exons.head()

,chrom,intron_start,intron_stop,strand,intron_motif,annotated,intron_location,exon_stop,exon_start,exon_5p,exon_3p
0,chr1,1309283,1309379,-,GT/AG,True,chr1:1309283-1309379:-,1309282,1309380,exon:chr1:1309380-1309825:-,"exon:chr1:1309110-1309282:-,exon:chr1:1309112-..."
1,chr1,1309826,1310084,-,GT/AG,True,chr1:1309826-1310084:-,1309825,1310085,"exon:chr1:1310085-1310537:-,exon:chr1:1310085-...",exon:chr1:1309380-1309825:-
2,chr1,1310171,1310533,-,GT/AG,True,chr1:1310171-1310533:-,1310170,1310534,exon:chr1:1310534-1310875:-,"exon:chr1:1310085-1310170:-,exon:chr1:1309624-..."
3,chr1,1337637,1341188,-,GT/AG,True,chr1:1337637-1341188:-,1337636,1341189,"exon:chr1:1341189-1341266:-,exon:chr1:1341189-...","exon:chr1:1337288-1337636:-,exon:chr1:1337293-..."
4,chr1,1341267,1342288,-,GT/AG,True,chr1:1341267-1342288:-,1341266,1342289,"exon:chr1:1342289-1342399:-,exon:chr1:1342289-...","exon:chr1:1341189-1341266:-,exon:chr1:1340489-..."


In [24]:
# sj_with_exons['junction_location'] = sj_with_exons.index.map(lambda x: x.split('|')[0])
# sj_with_exons.head()

In [7]:
# # %%time
# sj_with_exons_subset_3p = sj_with_exons_subset.drop_duplicates('exon_3p')
# grouped = sj_with_exons_subset_3p.groupby('intron_location')
# downstream_exons_subset = grouped.apply(lambda x: x.exon_3p.dropna().str.split(',').apply(pd.Series, 1))
# downstream_exons_subset = downstream_exons_subset.stack()
# downstream_exons_subset.index = downstream_exons_subset.index.droplevel((-2, -1))
# downstream_exons_subset = downstream_exons_subset.reset_index()
# downstream_exons_subset = downstream_exons_subset.rename(columns={0: 'exon'})
# downstream_exons_subset['direction'] = 'downstream'
# downstream_exons_subset.head()

,junction_location,exon,direction
0,chr2:74362786-74369398:-,exon:chr2:74362525-74362785:-,downstream
1,chr2:74362786-74369398:-,exon:chr2:74362528-74362785:-,downstream
2,chr2:74362786-74369398:-,exon:chr2:74362542-74362785:-,downstream
3,chr2:74369488-74372315:-,exon:chr2:74369399-74369487:-,downstream
4,chr2:74372431-74374947:-,exon:chr2:74372316-74372430:-,downstream


In [8]:
# # %%time
# sj_with_exons_subset_5p = sj_with_exons_subset.drop_duplicates('exon_5p')
# grouped = sj_with_exons_subset_5p.groupby('intron_location')
# upstream_exons_subset = grouped.apply(lambda x: x.exon_5p.dropna().str.split(',').apply(pd.Series, 1))
# upstream_exons_subset = upstream_exons_subset.stack()
# upstream_exons_subset.index = upstream_exons_subset.index.droplevel((-2, -1))
# upstream_exons_subset = upstream_exons_subset.reset_index()
# upstream_exons_subset = upstream_exons_subset.rename(columns={0: 'exon'})
# upstream_exons_subset['direction'] = 'upstream'
# upstream_exons_subset.head()

,junction_location,exon,direction
0,chr2:74362786-74369398:-,exon:chr2:74369399-74369487:-,upstream
1,chr2:74362786-74372315:-,exon:chr2:74372316-74372430:-,upstream
2,chr2:74362786-74372315:-,exon:chr2:74372316-74372635:-,upstream
3,chr2:74362786-74372315:-,exon:chr2:74372316-74375039:-,upstream
4,chr2:74372431-74373368:-,exon:chr2:74373369-74374353:-,upstream


In [6]:
%%time
# sj_with_exons_5p = sj_with_exons.drop_duplicates('exon_5p')
grouped = sj_with_exons.groupby('intron_location')
upstream_exons = grouped.apply(lambda x: x.exon_5p.dropna().str.split(',').apply(pd.Series, 1))
upstream_exons = upstream_exons.stack()
upstream_exons.index = upstream_exons.index.droplevel((-2, -1))
upstream_exons = upstream_exons.reset_index()
upstream_exons = upstream_exons.rename(columns={0: 'exon'})
upstream_exons['direction'] = 'upstream'


CPU times: user 10min 20s, sys: 3.04 s, total: 10min 23s
Wall time: 10min 23s


In [7]:
upstream_exons.head()

,intron_location,exon,direction
0,chr10:100008749-100010821:-,exon:chr10:100010822-100010933:-,upstream
1,chr10:100010934-100011322:-,exon:chr10:100011323-100011459:-,upstream
2,chr10:100143626-100144703:-,exon:chr10:100144704-100144824:-,upstream
3,chr10:100144825-100146957:-,exon:chr10:100146958-100147064:-,upstream
4,chr10:100147065-100155147:-,exon:chr10:100155148-100155209:-,upstream


In [8]:
%%time
# sj_with_exons_3p = sj_with_exons.drop_duplicates('exon_3p')
grouped = sj_with_exons.groupby('intron_location')
downstream_exons = grouped.apply(lambda x: x.exon_3p.dropna().str.split(',').apply(pd.Series, 1))
downstream_exons = downstream_exons.stack()
downstream_exons.index = downstream_exons.index.droplevel((-2, -1))
downstream_exons = downstream_exons.reset_index()
downstream_exons = downstream_exons.rename(columns={0: 'exon'})
downstream_exons['direction'] = 'downstream'


CPU times: user 10min 38s, sys: 1.57 s, total: 10min 39s
Wall time: 10min 40s


In [9]:
downstream_exons.head()

,intron_location,exon,direction
0,chr10:100008749-100010821:-,exon:chr10:100007447-100008748:-,downstream
1,chr10:100010934-100011322:-,exon:chr10:100010822-100010933:-,downstream
2,chr10:100143626-100144703:-,exon:chr10:100143322-100143625:-,downstream
3,chr10:100144825-100146957:-,exon:chr10:100144704-100144824:-,downstream
4,chr10:100147065-100155147:-,exon:chr10:100146958-100147064:-,downstream


In [10]:
upstream_exons.loc[upstream_exons.intron_location == 'chr12:110937352-110939853:-']

,intron_location,exon,direction
34245,chr12:110937352-110939853:-,exon:chr12:110939854-110939922:-,upstream
34246,chr12:110937352-110939853:-,exon:chr12:110939854-110939904:-,upstream
34247,chr12:110937352-110939853:-,exon:chr12:110939854-110939883:-,upstream
34248,chr12:110937352-110939853:-,exon:chr12:110939854-110939897:-,upstream
34249,chr12:110937352-110939853:-,exon:chr12:110939854-110939916:-,upstream
34250,chr12:110937352-110939853:-,exon:chr12:110939854-110939913:-,upstream
34251,chr12:110937352-110939853:-,exon:chr12:110939854-110939912:-,upstream
34252,chr12:110937352-110939853:-,exon:chr12:110939854-110939914:-,upstream


In [11]:
junction_exons = pd.concat([upstream_exons, downstream_exons])
junction_exons.head()

,intron_location,exon,direction
0,chr10:100008749-100010821:-,exon:chr10:100010822-100010933:-,upstream
1,chr10:100010934-100011322:-,exon:chr10:100011323-100011459:-,upstream
2,chr10:100143626-100144703:-,exon:chr10:100144704-100144824:-,upstream
3,chr10:100144825-100146957:-,exon:chr10:100146958-100147064:-,upstream
4,chr10:100147065-100155147:-,exon:chr10:100155148-100155209:-,upstream


In [12]:
junction_exons.shape

(659005, 3)

In [13]:
junction_exons.drop_duplicates().shape

(659005, 3)

In [15]:
junction_exons = junction_exons.drop_duplicates()

In [16]:
junction_exons.shape

(659005, 3)

In [17]:
junction_exons.to_csv('{}/junction_exon_triples.csv'.format(folder))